# Matching `summary(lm(...))` output

These are my notes on matching the report that is printed when you call `summary` on a linear model in R.

We will use the `trees` dataset. `Girth` is the tree diameter in inches, `Height` is the tree height in feet, `Volume` is the amount of timber in cubic feet. 

## Univariate case

In [1]:
data(trees)
model1 <- lm(Volume ~ Girth, data = trees)
summary(model1)


Call:
lm(formula = Volume ~ Girth, data = trees)

Residuals:
   Min     1Q Median     3Q    Max 
-8.065 -3.107  0.152  3.495  9.587 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -36.9435     3.3651  -10.98 7.62e-12 ***
Girth         5.0659     0.2474   20.48  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.252 on 29 degrees of freedom
Multiple R-squared:  0.9353,	Adjusted R-squared:  0.9331 
F-statistic: 419.4 on 1 and 29 DF,  p-value: < 2.2e-16


### Residuals

The $\hat{y_i}$ are found in the `fitted.values` field. The residuals $y_i - \hat{y_i}$ are in the `residuals` field. Each can be extracted using the `fitted` and `residuals` functions. We can get the residual quantiles by using either one.

In [2]:
round(quantile(trees$Volume - fitted(model1)), 3)

0%    25%    50%    75%   100% 
-8.065 -3.107  0.152  3.495  9.587

In [3]:
round(quantile(residuals(model1)), 3)

0%    25%    50%    75%   100% 
-8.065 -3.107  0.152  3.495  9.587

### Coefficients

Coefficients are found by $\beta = (X^T X)^{-1} X^T y$.

In [4]:
X <- model.matrix(~ Girth, data = trees)
beta <- round(solve(t(X) %*% X) %*% t(X) %*% trees$Volume, 4)
beta

(Intercept),-36.9435
Girth,5.0659


Standard errors can be found via $\sqrt{diag(X^T X)^{-1}} \sigma$.

In [5]:
se <- round(sqrt(diag(solve(t(X) %*% X))) * sqrt(sum(model1$res^2) / (nrow(trees)-ncol(X))), 4)
se

(Intercept)       Girth 
     3.3651      0.2474

The `t` values are just the coefficients divided by the standard errors.

In [6]:
round(beta / se, 2)

(Intercept),-10.98
Girth,20.48


The `p` values are found by looking up the probability distribution. The degrees of freedom is observation count minus number of parameters (where we include the intercept), and multiply by two to make the test two sided.

In [7]:
pt(-abs(beta / se), nrow(trees) - ncol(X))*2

(Intercept),7.618853e-12
Girth,8.664230e-19


### Residual standard error

Residual standard error is the `sd` of the residuals adjusted for degrees of freedom. Degrees of freedom here is number of observations (31) minus number of coeficients (2).

In [8]:
round(sqrt(sum(model1$res^2) / (nrow(trees)-ncol(X))), 3)

[1] 4.252

In [9]:
c(df = nrow(trees) - 2)

df 
29

Compare to acutal `sd` of residuals.

In [10]:
sd(model1$residuals)

[1] 4.18052

### R-squared

Total sum of squares (TSS) is $\sum_i {(y_i - \bar{y})^2}$, the total variation in the dataset adjusted for the mean. Residual sum of squares (RSS) is $\sum_i {(y_i - \hat{y_i})^2}$, the variation in the prediction. $R^2$, the coefficient of determination, is $1 - \frac{RSS}{TSS}$. This is the amount of explained variation. The explained sum of squares (ESS) is $\sum_i {(\hat{y_i} - \bar{y})^2}$. It is always the case that $TSS = ESS + RSS$.

For multiple regression, the formulas need to be put in matrix terms, see [Wikipedia](https://en.wikipedia.org/wiki/Explained_sum_of_squares). It is still the case that $TSS = ESS + RSS$ if a certain condition holds which (from what I can tell) it always does when using ordinary least squares.

In [11]:
TSS <- sum((trees$Volume - mean(trees$Volume))^2)
RSS <- sum(model1$residuals^2)
ESS <- sum((model1$fitted.values - mean(trees$Volume))^2)
c(TSS = TSS, ESS = ESS, RSS = RSS, `ESS+RSS` = ESS + RSS, r2 = 1 - RSS / TSS)

TSS          ESS          RSS      ESS+RSS           r2 
8106.0838710 7581.7813323  524.3025387 8106.0838710    0.9353199

Adjusted $R^2$ adds a penalty for number of predictors, $Adj. R^2 = 1 - \frac{RSS/(n-p-1)}{TSS/(n-1)} = 1 - \left(\frac{RSS}{TSS}\right)\left(\frac{n-1}{n-p-1}\right)$. Here, $p$ is the number of predictors and 1 reflects the constant term so the divisors of the sums of squares are just the degrees of freedom for each. As $p$ increases, $n-p-1$ decreases and hence the penalty $\frac{n-1}{n-p-1}$ increases.

In [12]:
1 - (RSS/TSS)*((nrow(trees)-1)/(nrow(trees)-1-1))

[1] 0.9330895

An ANOVA table gives the same information as well as the F-statistic from the bottom of the `summary` output.

In [13]:
round(anova(model1), 2)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
Girth,1,7581.78,7581.78,419.36,0
Residuals,29,524.30,18.08,NA,NA


In [14]:
(ESS / 1) / (RSS / 29)

[1] 419.3603

## Multi-variate case

In [15]:
model2 <- lm(Volume ~ Girth + Height, data = trees)
summary(model2)


Call:
lm(formula = Volume ~ Girth + Height, data = trees)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.4065 -2.6493 -0.2876  2.2003  8.4847 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -57.9877     8.6382  -6.713 2.75e-07 ***
Girth         4.7082     0.2643  17.816  < 2e-16 ***
Height        0.3393     0.1302   2.607   0.0145 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.882 on 28 degrees of freedom
Multiple R-squared:  0.948,	Adjusted R-squared:  0.9442 
F-statistic:   255 on 2 and 28 DF,  p-value: < 2.2e-16


### Residuals

Just like the univariate case.

In [16]:
round(quantile(trees$Volume - fitted(model2)), 4)

0%     25%     50%     75%    100% 
-6.4065 -2.6493 -0.2876  2.2003  8.4847

### Coefficients

Coefficients are found by $\beta = (X^T X)^{-1} X^T y$.

In [17]:
X <- model.matrix(~ Girth + Height, data = trees)
beta <- round(solve(t(X) %*% X) %*% t(X) %*% trees$Volume, 4)
beta

(Intercept),-57.9877
Girth,4.7082
Height,0.3393


Standard errors can be found via $\sqrt{diag(X^T X)^{-1}} \sigma$ but make sure to take off another degree of freedom.

In [18]:
se <- round(sqrt(diag(solve(t(X) %*% X))) * sqrt(sum(model2$res^2) / (nrow(trees)-ncol(X))), 4)
se

(Intercept)       Girth      Height 
     8.6382      0.2643      0.1302

The `t` values are just the coefficients divided by the standard errors.

In [19]:
round(beta / se, ncol(X))

(Intercept),-6.713
Girth,17.814
Height,2.606


The `p` values are found by looking up the probability distribution. The degrees of freedom is observation count minus number of parameters (where we include the intercept), and multiply by two to make the test two sided.

In [20]:
pt(-abs(beta / se), nrow(trees) - ncol(X))*2

(Intercept),2.749328e-07
Girth,8.249983e-17
Height,1.451141e-02


### Residual standard error

Residual standard error is the `sd` of the residuals adjusted for degrees of freedom. Degrees of freedom here is number of observations (31) minus number of coeficients (3).

In [21]:
round(sqrt(sum(model2$res^2) / (nrow(trees)-ncol(X))), 3)

[1] 3.882

In [22]:
c(df = nrow(trees) - ncol(X))

df 
28

Compare to acutal `sd` of residuals.

In [23]:
sd(residuals(model2))

[1] 3.750206

## Session info

In [24]:
Sys.Date()

[1] "2017-05-25"

In [25]:
sessionInfo()

R version 3.3.3 (2017-03-06)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.2 LTS

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] R6_2.2.1            magrittr_1.5        IRdisplay_0.4.4    
 [4] pbdZMQ_0.2-6        tools_3.3.3         crayon_1.3.2       
 [7] uuid_0.1-2          stringi_1.1.5       IRkernel_0.8.6.9000
[10] jsonlite_1.4        stringr_1.2.0       digest_0.6.12      
[13] repr_0.12.0         evaluate_0.10      